In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from litQeval.eval_utils import get_data, evaluate_recall, fscore
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

In [13]:
baseline = '"Cervical Myelopathy"'
predicted = """
"spinal stenosis" OR "spinal surgery risks" OR "cervical myelopathy" OR "gait disturbance" OR "corticosteroid injections" OR "degenerative disc disease" OR "surgery for myelopathy" OR "myelopathy symptoms" OR "nerve root compression" OR "upper limb weakness" OR "mri cervical spine" OR "cervical radiculopathy" OR "cervical spondylotic myelopathy" OR "myelopathy assessment" OR 
(("rehabilitation" OR "long-term outcomes" OR "patient education" OR "pain management" OR "chronic pain" OR "physical therapy" OR "neurological symptoms" OR "postoperative care" OR "cervical spine" OR "neck pain" OR "reflex changes" OR "mobility issues" OR "spinal cord compression" OR "electromyography (emg)") AND ("Cervical Myelopathy"))
"""

data = get_data(baseline, predicted)
core_pubs = data["core_pubs"]
core_mean_embedding = data["core_mean_embedding"]
baseline_pubs = data["baseline_pubs"]
predicted_pubs = data["predicted_pubs"]
baseline_vs = data["baseline_vs"]
predicted_vs = data["predicted_vs"]
core_vs = data["core_vs"]
predicted_embeddings = np.array([embedding for embedding in predicted_vs.get(include=["embeddings"])["embeddings"]])
baseline_embeddings = np.array([embedding for embedding in baseline_vs.get(include=["embeddings"])["embeddings"]])
core_embeddings = np.squeeze([core_vs.get(i,include=["embeddings"])["embeddings"] for i in core_pubs])


In [14]:
core_mean_embedding.reshape(1, -1).shape, predicted_embeddings.shape, baseline_embeddings.shape

((1, 1536), (37068, 1536), (4298, 1536))

In [15]:
# predicted
cosine_sim = cosine_similarity(core_mean_embedding, predicted_embeddings).flatten()
predicted_pubs["similarity"] = cosine_sim

core_pubs_in_predicted = predicted_pubs[predicted_pubs["id"].isin(core_pubs)]
threshold = core_pubs_in_predicted["similarity"].min()
relevant_predicted_pubs = predicted_pubs[predicted_pubs["similarity"] >= threshold].copy()
relevant_predicted_pubs.shape[0] 

11719

In [16]:
# baseline
cosine_sim = cosine_similarity(core_mean_embedding, baseline_embeddings).flatten()
baseline_pubs["similarity"] = cosine_sim

core_pubs_in_baseline = baseline_pubs[baseline_pubs["id"].isin(core_pubs)]
threshold = core_pubs_in_baseline["similarity"].min()
relevent_baseline_pubs = baseline_pubs[baseline_pubs["similarity"] >= threshold].copy()
relevent_baseline_pubs.shape[0]

2249

### Cosine Similarity Measure

In [17]:
recall = evaluate_recall(core_pubs, baseline_pubs, predicted_pubs)
# semnatic precision: every element that is more similar than the least similar core publication is considered relevant
# relevant_predicted_pubs: publications that are more similar than the least similar core publication.
pred_precision = relevant_predicted_pubs.shape[0] / predicted_pubs.shape[0] # total number of found publications
baseline_precision = relevent_baseline_pubs.shape[0] / baseline_pubs.shape[0]
pred_f2 = fscore(pred_precision, recall["predicted_recall"], 2)
baseline_f2 = fscore(baseline_precision, recall["baseline_recall"], 2)
df = pd.DataFrame({
    "Semantic Precision": [pred_precision, baseline_precision],
    "Recall": [recall["predicted_recall"], recall["baseline_recall"]],
    "Semantic F2": [pred_f2, baseline_f2]
}, index=["Predicted", "Baseline"])
df

,Semantic Precision,Recall,Semantic F2
Predicted,0.316149,0.723404,0.575210
Baseline,0.523267,0.276596,0.305388


### Minimum Volume Enclosing Ellipsoid MMVE

In [18]:
import numpy.linalg as la

def mvee(points, tol=0.0001):
    """
    Finds the ellipse equation in "center form"
    (x-c).T * A * (x-c) = 1
    """
    N, d = points.shape
    Q = np.column_stack((points, np.ones(N))).T
    err = tol+1.0
    u = np.ones(N)/N
    while err > tol:
        # assert u.sum() == 1 # invariant
        X = np.dot(np.dot(Q, np.diag(u)), Q.T)
        M = np.diag(np.dot(np.dot(Q.T, la.inv(X)), Q))
        jdx = np.argmax(M)
        step_size = (M[jdx]-d-1.0)/((d+1)*(M[jdx]-1.0))
        new_u = (1-step_size)*u
        new_u[jdx] += step_size
        err = la.norm(new_u-u)
        u = new_u
    c = np.dot(u, points)
    A = la.inv(np.dot(np.dot(points.T, np.diag(u)), points)
               - np.multiply.outer(c, c))/d
    return A, c

A, c = mvee(core_embeddings)

In [19]:
# point_wise
def is_inside_ellipse(A, c, points):
    return np.array([((point - c) @ A @ (point - c).T) <= 1 for point in tqdm(points)])

# if we have enough memory
def is_inside_ellipse_v2(A, c, points):
    shifted_points = points - c
    return np.diag((shifted_points @ A) @ shifted_points.T <= 1)

base_is_inside = is_inside_ellipse(A, c, baseline_embeddings)
predicted_is_inside = is_inside_ellipse(A, c, predicted_embeddings)

  0%|          | 0/4298 [00:00<?, ?it/s]

  0%|          | 0/37068 [00:00<?, ?it/s]

In [20]:
mvve_prec_baseline = base_is_inside.sum() / len(base_is_inside)
mvve_prec_predicted = predicted_is_inside.sum() / len(predicted_is_inside)

mvve_df = pd.DataFrame({
    "MVVE Precision": [mvve_prec_predicted, mvve_prec_baseline],
    "Recall": [recall["predicted_recall"], recall["baseline_recall"]],
    "MVVE F2": [fscore(mvve_prec_predicted, recall["predicted_recall"], 2), fscore(mvve_prec_baseline, recall["baseline_recall"], 2)]
}, index=["Predicted", "Baseline"])
mvve_df

,MVVE Precision,Recall,MVVE F2
Predicted,0.437655,0.723404,0.639851
Baseline,0.488599,0.276596,0.302880


In [66]:
results = pd.DataFrame({
    "Query": [predicted] + [baseline],
    "Recall": [recall["baseline_recall"], recall["predicted_recall"]],
    "Semantic Precision": [baseline_precision, pred_precision],
    "Semantic F2": [baseline_f2, pred_f2],
    "MVVE Precision": [mvve_prec_predicted, mvve_prec_baseline],
    "MVVE F2": [fscore(mvve_prec_predicted, recall["predicted_recall"], 2), fscore(mvve_prec_baseline, recall["baseline_recall"], 2)]
}, index=["Predicted", "Baseline"])

try:
    old_results = pd.read_excel("results.xlsx", index_col=0)
    results = pd.concat([old_results, results]).drop_duplicates(subset=["Query"])
    results.to_excel("results.xlsx")
    display(results)
except FileNotFoundError:
    results.to_excel("results.xlsx")


,Query,Recall,Semantic Precision,Semantic F2,MVVE Precision,MVVE F2
Predicted,"\n""spinal stenosis"" OR ""spinal surgery risks"" ...",0.276596,0.523267,0.305388,0.437655,0.639851
Baseline,"""Cervical Myelopathy""",0.723404,0.316149,0.575210,0.488599,0.302880
